# IMPORT

In [1]:
!pip -q install kaggle
from google.colab import userdata

username = userdata.get('KAGGLE_USERNAME')
key = userdata.get('KAGGLE_KEY')

!mkdir -p ~/.kaggle
!echo '{{"username":"{username}","key":"{key}"}}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle competitions download -c financial-analysis-agent

!mkdir -p dataset/
!unzip -q financial-analysis-agent.zip -d dataset/

  0% 0.00/268k [00:00<?, ?B/s]
100% 268k/268k [00:00<00:00, 393MB/s]


In [3]:
import pandas as pd

In [7]:
from google import genai

client = genai.Client(api_key=userdata.get('GEMINI_API'))

In [5]:
from tqdm import tqdm

In [6]:
import google.generativeai as genai
from google.colab import userdata

# Zero shot time

In [8]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words",
)

print(response.text)

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_input_token_count', 'quotaId': 'GenerateContentInputTokensPerModelPerDay-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-pro'}}, {'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_input_token_count', 'quotaId': 'GenerateContentInputTokensPerModelPerMinute-FreeTier', 'quotaDimensions': {'model': 'gemini-2.5-pro', 'location': 'global'}}, {'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimen

In [19]:
test = pd.read_csv("/content/dataset/test.csv")
test

,id,query
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,Answer the question with the appropriate optio...
2,8a722080-bc16-49db-89c9-100cd61cd28a,Answer the question with the appropriate optio...
3,75316e95-88f4-4fef-83b9-dde0aa52889a,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
4,bcca13bc-2675-4645-82cc-7e4c412ed294,"Given the data and tweets, could you project w..."
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,Answer the question with the appropriate optio...
495,73c720b5-1101-4790-af52-3366823e1d32,"ตอบคำถามโดยใช้ตัวเลือกที่เหมาะสม A, B, C และ D..."
496,357db18f-d872-416e-a07f-753099853d9c,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
497,2d8b1419-1c46-4e83-892a-081fb417de38,Scrutinize the data and tweets to envisage if ...


In [73]:
for idx, row in tqdm(test[:].iterrows()):
  response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Answer this without any explination : "+row['query'],
  )
  print(response.text)
  test.loc[idx, 'answer'] = response.text
  print(f'Currently at {idx}')

0it [00:00, ?it/s]


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-flash'}, 'quotaValue': '250'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '23s'}]}}

In [122]:
test.isna().sum()

,0
id,0
query,0
answer,0


In [101]:
sub = pd.read_csv("/content/dataset/submission.csv")
sub

,id,answer
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,D
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,B
2,8a722080-bc16-49db-89c9-100cd61cd28a,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,NaN
4,bcca13bc-2675-4645-82cc-7e4c412ed294,NaN
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,NaN
495,73c720b5-1101-4790-af52-3366823e1d32,NaN
496,357db18f-d872-416e-a07f-753099853d9c,NaN
497,2d8b1419-1c46-4e83-892a-081fb417de38,NaN


In [124]:
dummy = test.copy()

In [125]:
dummy = dummy[['id', 'answer']]

In [126]:
for idx, row in dummy.iterrows():
  row['answer'] = row['answer'][0]
  if row['answer'] == 'เ' or row['answer'] == 'ไ':
    print(idx)

277
355


In [127]:
test.iloc[355]

,355
id,cde3aba2-c5a4-47ec-835c-92f9b062c4af
query,ประเมินข้อมูลและทวีตเพื่อประมาณว่าราคาปิดของ $...
answer,เพิ่มขึ้น


In [129]:
to_replace = {'R' : 'Rise', 'F' : 'Fall', 'ข' : 'Rise', 'ล' : 'Fall', 'เ':'Rise'}

In [130]:
dummy['answer'] = dummy['answer'].replace(to_replace)

In [131]:
dummy['answer'].value_counts()

,count
answer,
D,115
C,102
B,99
A,82
Rise,66
Fall,35


In [133]:
dummy.to_csv("2.5-FlashAPI.csv", index=False)

In [135]:
dummy

,id,answer
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,D
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,B
2,8a722080-bc16-49db-89c9-100cd61cd28a,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,A
4,bcca13bc-2675-4645-82cc-7e4c412ed294,Rise
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,D
495,73c720b5-1101-4790-af52-3366823e1d32,A
496,357db18f-d872-416e-a07f-753099853d9c,D
497,2d8b1419-1c46-4e83-892a-081fb417de38,Fall
